In [1]:
from numcosmo_py import Nc, Ncm, sky_match
Ncm.cfg_init()

from astropy.table import Table, join, unique
import pandas as pd

import sys
sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
import GCRCatalogs
print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

%load_ext autoreload
%autoreload 2

GCRCatalogs = 1.10.2 | GCR = 0.9.2


## Catalogs

In [2]:
#Cosmo DC2
cosmodc2 = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

#Cosmo DC2 + RedMaPPer
catalog_dc2_redmapper = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

print("CosmoDC2 info:")
print(f"Version: {cosmodc2.get_catalog_info('version')}")
print(f"Sky area: {cosmodc2.get_catalog_info('sky_area')}")
print(f"Cosmology: {cosmodc2.get_catalog_info('cosmology')}")

# quantities = catalog_dc2_redmapper.list_all_quantities()
# cluster_quantities = [q for q in quantities if 'member' not in q]
# member_quantities = [q for q in quantities if 'member' in q]
    
# cluster_data = Table(catalog_dc2_redmapper.get_quantities(cluster_quantities)) # cDC2 + RM cluster data
# member_data = Table(catalog_dc2_redmapper.get_quantities(member_quantities)) # cDC2 + RM member data

# # Restrictions on mass and redshift
# min_halo_mass = 1e13 #Msun
# redshift_max = 1.2

# quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']
# query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +') & (is_central < ' + str(redshift_max) +') ')
# truth_data = Table(cosmodc2.get_quantities(quantities_wanted, [query]))

# table_halos = truth_data[truth_data['is_central'] == True]

CosmoDC2 info:
Version: 1.1.4
Sky area: 439.78987
Cosmology: {'H0': 71.0, 'Om0': 0.2648, 'Ob0': 0.0448, 'sigma8': 0.8, 'n_s': 0.963}


In [3]:
# CLUSTER DATA

cluster_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/cluster_data.fits')

cluster_member_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/member_data.fits')
cluster_member_data.rename_column('cluster_id_member', 'cluster_id')


In [4]:
# HALO DATA

truth_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/truth_data.fits')

truthdf = truth_data.to_pandas()

hascentral = truthdf[truthdf['is_central'] ==  True]
halo_central_ids = list(hascentral['halo_id'].unique())
central_position = hascentral[['ra', 'dec', 'redshift', 'halo_id']].rename(columns = {'ra': 'halo_ra', 'dec': 'halo_dec', 'redshift': 'halo_redshift'})
halos_dataframe = pd.merge(truthdf[truthdf['halo_id'].isin(halo_central_ids)], central_position, how='inner', on=['halo_id'])


halo_member_data = Table.from_pandas(halos_dataframe)
halo_data =  unique(halo_member_data['halo_id', 'halo_mass', 'halo_ra', 'halo_dec', 'halo_redshift'])

## Matching

In [5]:
halo_coordinates = {"RA":"halo_ra" , "DEC":"halo_dec" , "z":"halo_redshift"}
halo_ids  = {"ID": "halo_id", "MemberID": "galaxy_id"}

detections_coordinates =  {"RA":"cluster_ra" , "DEC":"cluster_dec" , "z":"cluster_redshift"}
detections_ids  = {"ID": "cluster_id", "MemberID": "id_member"}


halos = sky_match.SkyMatch(
    query_data= halo_data,
    query_coordinates=halo_coordinates,
    query_member_data=truth_data,
    query_ids = halo_ids, 
    match_data=cluster_data,
    match_coordinates=detections_coordinates,
    match_member_data=cluster_member_data,
    match_ids = detections_ids
)

detections = halos.invert_query_match()

In [6]:
%time halos_matched = halos.match_ID()

CPU times: user 14.6 s, sys: 564 ms, total: 15.1 s
Wall time: 15.2 s


In [7]:
%time best = halos_matched.select_best()

CPU times: user 5.57 s, sys: 80.4 ms, total: 5.65 s
Wall time: 5.67 s


In [8]:
%time best_table = halos_matched.to_table_best(best)

CPU times: user 8.66 ms, sys: 139 μs, total: 8.8 ms
Wall time: 7.88 ms


In [9]:
best_table

Index,ID,RA,DEC,z,Index_matched,ID_matched,RA_matched,DEC_matched,z_matched
int64,int64,float64,float64,float64,int64,int32,float64,float64,float32
6,80442,57.94542121163374,-33.78088245138828,0.09851626146477921,20752,29091,57.94542121163374,-33.78088245138828,0.11098104
14,181293,69.18918616334467,-43.41185729310556,0.6914193960527157,44743,54348,69.23231434019651,-43.40737480896762,0.5958041
28,100094347,56.02076522849643,-34.111500439861594,0.42810309720887485,20348,17063,56.02076522849643,-34.111500439861594,0.42926645
37,100163315,48.50744045376903,-41.83517844846393,0.5582766003633479,31407,20166,48.4881969046676,-41.83130719633555,0.55161095
38,100172347,74.26408742497946,-41.6114369604841,0.4113409840633473,40195,15005,74.25846814562257,-41.60172145684775,0.4096808
44,200036315,68.74911850604337,-28.147026057439266,0.5530162506288185,2615,14597,68.74911850604337,-28.147026057439266,0.55668956
48,200052315,51.605986269838105,-29.60232739710236,0.5618385735667621,6613,43716,51.605986269838105,-29.60232739710236,0.55545753
59,200154300,58.9473953940489,-41.58992359526981,0.620089839910746,38915,55247,58.9473953940489,-41.58992359526981,0.60096467
61,200164272,51.436015064672446,-41.75209830210666,0.8064893683349668,33206,49737,51.46094017014448,-41.75691780432012,0.82503825


In [10]:
%time complete_table = halos_matched.to_table_complete()

CPU times: user 11.2 s, sys: 668 ms, total: 11.9 s
Wall time: 11.9 s


In [11]:
complete_table

Index,RA,DEC,z,Index_matched,linking coefficient,RA_matched,DEC_matched,z_matched
int64,float64,float64,float64,object,object,object,object,object
0,62.754892124696084,-27.083535313771,0.23477334600242505,[],[],cluster_ra\n----------,cluster_dec\n-----------,cluster_redshift\n----------------
1,51.1179679800395,-28.372045175477723,1.0548603701287829,[],[],cluster_ra\n----------,cluster_dec\n-----------,cluster_redshift\n----------------
2,64.63302904423612,-28.26120432574449,1.0525068935128097,[],[],cluster_ra\n----------,cluster_dec\n-----------,cluster_redshift\n----------------
3,68.8258628075322,-29.652783978089786,1.1945141599537932,[],[],cluster_ra\n----------,cluster_dec\n-----------,cluster_redshift\n----------------
4,74.95480242669737,-30.443213212439307,0.4636950460205296,[],[],cluster_ra\n----------,cluster_dec\n-----------,cluster_redshift\n----------------
5,51.8855768308472,-33.46608510006379,0.8833312616768676,[5071],[0.00042834],cluster_ra \n----------------\n51.8855768308472,cluster_dec \n------------------\n-33.46608510006379,cluster_redshift\n----------------\n 0.8949706
6,57.94542121163374,-33.78088245138828,0.09851626146477921,[20752],[0.07504806],cluster_ra \n-----------------\n57.94542121163374,cluster_dec \n------------------\n-33.78088245138828,cluster_redshift\n----------------\n 0.11098104
7,73.56006583740744,-34.8048982806667,0.6413555677689318,[],[],cluster_ra\n----------,cluster_dec\n-----------,cluster_redshift\n----------------
8,55.37265585300007,-35.05240527297359,0.34573518660213587,[],[],cluster_ra\n----------,cluster_dec\n-----------,cluster_redshift\n----------------


In [12]:
%time detections_matched = detections.match_ID()

CPU times: user 11.2 s, sys: 412 ms, total: 11.7 s
Wall time: 11.7 s


In [13]:
# detections_best = detections_matched.select_best()
# detections_matched.to_table_best(detections_best)

In [14]:
%time complete_table_detections = detections_matched.to_table_complete()

CPU times: user 2.63 s, sys: 151 ms, total: 2.79 s
Wall time: 2.83 s


In [15]:
complete_table_detections

Index,RA,DEC,z,Index_matched,linking coefficient,RA_matched,DEC_matched,z_matched
int64,float64,float64,float32,object,object,object,object,object
0,52.520436430812914,-25.717996124671657,0.5135129,[ 14975 35175 64795 94413 99355 120524 141898 157546],[1.16939338e+00 4.37684136e-04 4.02915987e-05 4.24701561e-03\n 3.07340567e-03 1.12441671e-04 6.46216499e-05 4.93165223e-04],halo_ra \n------------------\n 52.47252472078352\n52.519490104517175\n 52.4478531974849\n 52.59576331713594\n 52.51910078836549\n 52.55754534716189\n52.520436430812914\n 52.46679384145919,halo_dec \n-------------------\n-25.716805591145693\n -25.71783277825776\n-25.661437475452676\n-25.734587202373465\n-25.727465056757442\n-25.723195072440273\n-25.717996124671657\n -25.73146364501913,halo_redshift \n------------------\n 0.508253855698195\n0.5130878312551626\n0.5126658753616595\n0.5445493920357991\n 0.568313113431514\n0.5435103949284432\n0.5107235191306336\n 0.549872171611419
1,56.920376528125075,-27.20434071042304,0.5712697,[81173],[1.57552198e-05],halo_ra \n------------------\n56.920376528125075,halo_dec \n------------------\n-27.20434071042304,halo_redshift \n------------------\n0.5740296938561777
2,56.99849698116052,-26.7172081871169,1.040338,[145976 220929 234518],[0.00073353 0.00115996 0.00778767],halo_ra \n-----------------\n56.94681141114094\n56.99849698116052\n56.98989225804766,halo_dec \n-------------------\n-26.695797419139687\n -26.7172081871169\n-26.719138984756995,halo_redshift \n------------------\n0.9007398371775617\n 1.034775804769664\n 1.039629648645699
3,58.56362402841767,-27.429961828331713,0.63518524,[ 1443 4789 151063],[2.07519531e-03 1.06336806e+00 2.79017857e-04],halo_ra \n------------------\n 58.5804702698203\n58.642134930674025\n 58.56362402841767,halo_dec \n-------------------\n -27.37260768263168\n-27.447888117177566\n-27.429961828331713,halo_redshift \n------------------\n0.6412580532763981\n0.6493675318621106\n0.6426174147729209
4,58.35519782866568,-26.867687128773035,0.44230303,[ 28217 40641 57025 82880 101564 106337],[2.80023781e-01 1.08034653e-03 4.16479272e-05 3.65526049e-05\n 5.66552424e-03 4.18648434e-03],halo_ra \n------------------\n 58.41234410391299\n58.330658600210676\n 58.35519782866568\n 58.34992458608299\n 58.42789425170504\n 58.36699173349363,halo_dec \n-------------------\n-26.830799499352946\n-26.892740062428945\n-26.867687128773035\n-26.866270557202213\n-26.842405590535368\n -26.91741677749035,halo_redshift \n-------------------\n0.43926306621223987\n0.43145919838830626\n0.43919736152180566\n 0.4385807111345399\n 0.4403458740215409\n0.44729583107630044
5,57.76575707007693,-28.119101936484828,0.46697217,[ 484 70229 125986],[9.15936542e+00 3.59405525e-04 2.80974955e-03],halo_ra \n-----------------\n 57.8324889411615\n57.76575707007693\n57.87113442996517,halo_dec \n-------------------\n-28.110915524971148\n-28.119101936484828\n-28.156552136942338,halo_redshift \n-------------------\n0.49491395758100976\n 0.4680865186324199\n0.47633056873511315
6,58.509466974185045,-26.989676182075844,0.4413634,[27483 66831],[3.76720675e-05 7.20844812e-04],halo_ra \n------------------\n58.509466974185045\n 58.56727031647663,halo_dec \n-------------------\n-26.989676182075844\n-27.021981130067523,halo_redshift \n------------------\n0.4434960915528956\n0.3870391892857903
7,54.494027152041674,-27.896589555863095,1.0629631,[217828 236143],[0.00885032 0.00125245],halo_ra \n------------------\n54.467852730826095\n54.496631285075914,halo_dec \n-------------------\n-27.935083771961377\n -27.89888605332762,halo_redshift \n------------------\n1.1196150797169055\n1.0722671522570981
8,58.137368555958155,-27.199784884537003,1.037697,[147534 170079 200315 215590],[1.95407450e-02 2.60490881e-04 8.58463336e-05 5.22985537e-03],halo_ra \n------------------\n 58.1413424223981\n 58.16136201650431\n58.137368555958155\n 58.16083427111327,halo_dec \n-------------------\n -27.16945223934122\n -27.25292585956248\n-27.199784884537003\n-27.207224016845945,hal